# Keras DSVM


## Introduction

This recipe shows how to run Keras using Batch AI on DSVM. DSVM supports tensorflow, cntk and theano backends for running Keras. Currently only tensorflow and cntk backends supports running on GPU.

## Details

- DSVM has Keras framework preinstalled;
- Standard keras sample script [mnist_cnn.py](https://raw.githubusercontent.com/fchollet/keras/master/examples/mnist_cnn.py) is used;
- The script downloads the standard MNIST Database on its own;
- Standard output of the job will be stored on Azure File Share.

## Instructions

### Install Dependencies and Create Configuration file.
Follow [instructions](/recipes) to install all dependencies and create configuration file.

### Read Configuration and Create Batch AI client

In [1]:
from __future__ import print_function

from datetime import datetime
import sys

from azure.storage.file import FileService
import azure.mgmt.batchai.models as models

# utilities.py contains helper functions used by different notebooks
sys.path.append('../../')
import utilities

cfg = utilities.Configuration('../../configuration.json')
client = utilities.create_batchai_client(cfg)

### Create File Share

For this example we will create a new File Share with name `batchaidsvmsample` under your storage account.

**Note** You don't need to create new file share for every cluster. We are doing this in this sample to simplify resource management for you.

In [2]:
azure_file_share_name = 'batchaisample'
service = FileService(cfg.storage_account_name, cfg.storage_account_key)
service.create_share(azure_file_share_name, fail_on_exist=False)
print('Done')

Done


No handlers could be found for logger "azure.storage.common.storageclient"


### Configure Compute Cluster

- For this example we will use a gpu cluster of `STANDARD_NC6` nodes. Number of nodes in the cluster is configured with `nodes_count` variable;
- We will mount file share at folder with name `afs`. Full path of this folder on a computer node will be `$AZ_BATCHAI_MOUNT_ROOT/afs`;
- We will call the cluster `nc6`;


So, the cluster will have the following parameters:

In [3]:
azure_file_share = 'afs'
nodes_count = 2
cluster_name = 'nc6'

volumes = models.MountVolumes(
    azure_file_shares=[
        models.AzureFileShareReference(
            account_name=cfg.storage_account_name,
            credentials=models.AzureStorageCredentialsInfo(
                account_key=cfg.storage_account_key),
            azure_file_url = 'https://{0}.file.core.windows.net/{1}'.format(
                cfg.storage_account_name, azure_file_share_name),
            relative_mount_path=azure_file_share)
    ]
)

parameters = models.ClusterCreateParameters(
    location=cfg.location,
    vm_size="STANDARD_NC6",
    virtual_machine_configuration=models.VirtualMachineConfiguration(
        image_reference=models.ImageReference(
            publisher="microsoft-ads",
            offer="linux-data-science-vm-ubuntu",
            sku="linuxdsvmubuntu",
            version="latest")),
    scale_settings=models.ScaleSettings(
        manual=models.ManualScaleSettings(target_node_count=nodes_count)
    ),
    node_setup=models.NodeSetup(
        mount_volumes=volumes
    ),
    user_account_settings=models.UserAccountSettings(
        admin_user_name=cfg.admin,
        admin_user_password=cfg.admin_password,
        admin_user_ssh_public_key=cfg.admin_ssh_key
    )
)

### Create Compute Cluster

In [4]:
_ = client.clusters.create(cfg.resource_group, cluster_name, parameters).result()

### Monitor Cluster Creation

utilities.py contains a helper function allowing to wait for the cluster to become available - all nodes are allocated and finished preparation.

In [5]:
cluster = client.clusters.get(cfg.resource_group, cluster_name)
utilities.print_cluster_status(cluster)

Cluster state: steady Target: 2; Allocated: 2; Idle: 2; Unusable: 0; Running: 0; Preparing: 0; Leaving: 0


### Deploy Sample Script and Configure the Input Directories

- Download original sample script:

In [6]:
sample_script_url = 'https://raw.githubusercontent.com/fchollet/keras/master/examples/mnist_cnn.py'
utilities.download_file(sample_script_url, 'mnist_cnn.py')

- For each job we will create a folder containing a copy of [train_mnist.py](https://github.com/chainer/chainer/blob/master/examples/mnist/train_mnist.py). This allows each job to have it's own copy of the sample script (in case you would like to change it).

In [7]:
keras_sample_dir = "KerasSamples"
service = FileService(cfg.storage_account_name, cfg.storage_account_key)
service.create_directory(
    azure_file_share_name, keras_sample_dir, fail_on_exist=False)
service.create_file_from_path(
    azure_file_share_name, keras_sample_dir, 'mnist_cnn.py', 'mnist_cnn.py')
print('Done')

Done


- The job needs to know where to find train_mnist.py script (the chainer will download MNIST dataset on its own). So, we will configure an input directory for the script:

In [8]:
input_directories = [
    models.InputDirectory(
        id='SCRIPT',
        path='$AZ_BATCHAI_MOUNT_ROOT/{0}/{1}'.format(azure_file_share, keras_sample_dir))
]

The job will be able to reference those directories using ```$AZ_BATCHAI_INPUT_SCRIPT``` environment variable.

### Configure Output Directories
We will store standard and error output of the job in File Share:

In [9]:
std_output_path_prefix = '$AZ_BATCHAI_MOUNT_ROOT/{0}'.format(azure_file_share)

### Configure Job

- Will use configured previously input and output directories;
- Will run standard `mnist_cnn.py` from SCRIPT input directory using custom framework;
- Keral will use theano backend; DSVM supports cntk, tensorflow and theano backends for keral, just change KERAS_BACKEND to "tensorflow" or "theano" to use corresponding backend. Note, theano backend will run on CPU. 
- Will output standard output and error streams to file share.


In [11]:
parameters = models.job_create_parameters.JobCreateParameters(
     location=cfg.location,
     cluster=models.ResourceId(id=cluster.id),
     node_count=1,
     input_directories=input_directories,
     std_out_err_path_prefix=std_output_path_prefix,
     custom_toolkit_settings = models.CustomToolkitSettings(
         command_line='KERAS_BACKEND=cntk python $AZ_BATCHAI_INPUT_SCRIPT/mnist_cnn.py'))

### Create a training Job and wait for Job completion


In [12]:
job_name = datetime.utcnow().strftime('keras_%m_%d_%Y_%H%M%S')
job = client.jobs.create(cfg.resource_group, job_name, parameters).result()
print('Created Job: {}'.format(job.name))

Created Job: keras_03_19_2018_200832


### Wait for Job to Finish
The job will start running when the cluster will have enought idle nodes. The following code waits for job to start running printing the cluster state. During job run, the code prints current content of stdout.txt.

**Note** Execution may take several minutes to complete.

In [13]:
utilities.wait_for_job_completion(client, cfg.resource_group, job_name, cluster_name, 'stdouterr', 'stdout.txt')

Cluster state: steady Target: 2; Allocated: 2; Idle: 1; Unusable: 0; Running: 1; Preparing: 0; Leaving: 0
Job state: running ExitCode: None
Waiting for job output to become available...

11493376/11490434 [==============================] - 0s 0us/step
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Train on 60000 samples, validate on 10000 samples
Epoch 1/12

60000/60000 [==============================] - 22s 362us/step - loss: 0.3261 - acc: 0.9016 - val_loss: 0.0748 - val_acc: 0.9767
Epoch 2/12

60000/60000 [==============================] - 14s 241us/step - loss: 0.1105 - acc: 0.9669 - val_loss: 0.0524 - val_acc: 0.9829
Epoch 3/12

15232/60000 [======>.......................] - ETA: 10s - loss: 0.0905 - acc: 0.9737


38272/60000 [==================>...........] - ETA: 5s - loss: 0.0891 - acc: 0.9737


60000/60000 [==============================] - 17s 285us/step - loss: 0.0844 - acc: 0.9752 - val_loss: 0.0459 - val_acc: 0.9840
Epoch 4/12

 7168/60000 [==>...........................] - ETA: 16s - loss: 0.0667 - acc: 0.9805


30464/60000 [==============>...............] - ETA: 9s - loss: 0.0700 - acc: 0.9794


57344/60000 [===========================>..] - ETA: 0s - loss: 0.0693 - acc: 0.9798


...

30592/60000 [==============>...............] - ETA: 7s - loss: 0.0380 - acc: 0.9891


55552/60000 [==========================>...] - ETA: 1s - loss: 0.0381 - acc: 0.9889


60000/60000 [==============================] - 18s 300us/step - loss: 0.0381 - acc: 0.9889 - val_loss: 0.0288 - val_acc: 0.9902
Epoch 12/12

22272/60000 [==========>...................] - ETA: 10s - loss: 0.0384 - acc: 0.9897


50432/60000 [========================>.....] - ETA: 2s - loss: 0.0365 - acc: 0.9896


60000/60000 [==============================] - 18s 295us/step - loss: 0.0367 - acc: 0.9893 - val_loss: 0.0306 - val_acc: 0.9888
Test loss: 0.0306497847026
Test accuracy: 0.9888
Job state: succeeded ExitCode: 0


### Download stdout.txt and stderr.txt files for the Job

In [14]:
files = client.jobs.list_output_files(cfg.resource_group, job_name,
                                      models.JobsListOutputFilesOptions(outputdirectoryid='stdouterr')) 
for f in list(files):
    if f.download_url:
        utilities.download_file(f.download_url, f.name)
print('All files downloaded')

All files downloaded


In [15]:
print('stdout.txt content:')
with open('stdout.txt') as f:
    print(f.read())

stdout.txt content:

11493376/11490434 [==============================] - 0s 0us/step
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Train on 60000 samples, validate on 10000 samples
Epoch 1/12

60000/60000 [==============================] - 22s 362us/step - loss: 0.3261 - acc: 0.9016 - val_loss: 0.0748 - val_acc: 0.9767
Epoch 2/12

60000/60000 [==============================] - 14s 241us/step - loss: 0.1105 - acc: 0.9669 - val_loss: 0.0524 - val_acc: 0.9829
Epoch 3/12

60000/60000 [==============================] - 17s 285us/step - loss: 0.0844 - acc: 0.9752 - val_loss: 0.0459 - val_acc: 0.9840
Epoch 4/12

60000/60000 [==============================] - 19s 323us/step - loss: 0.0690 - acc: 0.9800 - val_loss: 0.0360 - val_acc: 0.9881
Epoch 5/12

60000/60000 [==============================] - 18s 302us/step - loss: 0.0612 - acc: 0.9818 - val_loss: 0.0364 - val_acc: 0.9872
Epoch 6/12

60000/60000 [==============================] - 17s 286us/step - loss: 0.0534 - 

### Delete the Job

In [ ]:
_ = client.jobs.delete(cfg.resource_group, job_name)

### Delete the Cluster
When you are finished with the sample and don't want to submit any more jobs you can delete the cluster using the following code.

In [ ]:
_ = client.clusters.delete(cfg.resource_group, cluster_name)

### Delete File Share
When you are finished with the sample and don't want to submit any more jobs you can delete the file share completely with all files using the following code.

In [ ]:
service = FileService(cfg.storage_account_name, cfg.storage_account_key)
service.delete_share(azure_file_share_name)